In [5]:
import pandas as pd
from tracker_manu import*
import cv2
import os

In [6]:
from ultralytics import YOLO



In [8]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [11]:

list_name = ["MOT17-02-DPM","MOT17-04-DPM","MOT17-05-DPM","MOT17-09-DPM"]

In [2]:
import re
def get_frames_list(images_folder):
    frames_list = []
    for frame in os.listdir(images_folder):
        if frame.endswith(".jpg"):
            # Extract the numerical part of the filename
            number = int(re.search(r'\d+', frame).group())
            frames_list.append((number, os.path.join(images_folder, frame)))
    # Sort by the numerical part and get the paths
    frames_list = sorted(frames_list, key=lambda x: x[0])
    return [frame[1] for frame in frames_list]

In [12]:

def tracking_with_Manu(model_name):
    
    model = YOLO(model_name)
    tracker=Tracker()
    
    for dataset_name in list_name :
        print("---------------------------------")
        print(dataset_name)
        images_path = f"data\\MOT17\\train\{dataset_name}\\img1"
        # images_path = f"data\\frames"
        output_path = f"output\\{dataset_name}"
        output_track_eval = f"TrackEval\\data\\trackers\\mot_challenge\\MOT17-train\\MPNTrack_Manu_{model_name}\\data"
        
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        if not os.path.exists(output_track_eval):
            os.makedirs(output_track_eval)
        if not os.path.exists(os.path.join(output_path,f"pred/Manu_{model_name}")):
            os.makedirs(os.path.join(output_path, f"pred/Manu_{model_name}"))
        if not os.path.exists(os.path.join(output_path, f"img_pred/Manu_{model_name}")):
            os.makedirs(os.path.join(output_path, f"img_pred/Manu_{model_name}"))
            
        output_file_path = os.path.join(output_path, f"pred/Manu_{model_name}/{dataset_name}.txt")
        output_file_path_trackEval = os.path.join(output_track_eval, f"{dataset_name}.txt")
        output_path_imgs = os.path.join(output_path, f"img_pred/Manu_{model_name}")

        images_files = get_frames_list(images_path)
        
        

        tracks_data = [] 
        for i, image_file in enumerate(images_files):
            
            
            # Lire l'image
            frame = cv2.imread(image_file)

            # Faire des prédictions avec ByteTrack
            results = model.predict(frame)

            # Initialiser une liste pour stocker les coordonnées des objets d'intérêt
            list_bboxes = []

            # Traitement des prédictions
            for result in results:
                # Convertir les prédictions en DataFrame pandas
                df = pd.DataFrame(result.boxes.data.detach().cpu().numpy())

               
                # df = df[df[5] == 41]  # CUP

                # Itérer sur chaque ligne du DataFrame filtré
                for _, row in df.iterrows():
                    x1, y1, x2, y2 = map(int, row[:4])  # Coordonnées de la boîte englobante
                    
                    list_bboxes.append([x1, y1, int(x2) ,  int(y2)])

            # Suivi des objets avec le tracker
            bbox_id = tracker.update(list_bboxes)

            # Dessiner les boîtes englobantes et les ID de piste sur l'image
            for bbox in bbox_id:
                x1, y1, x2, y2, track_id = bbox
                width = int(x2) - int(x1)  # width = x2 - x1
                height = int(y2) - int(y1)  # height = y2 - y1
                tracks_data.append((i, track_id, x1, y1, width, height,-1,-1,-1,-1))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, str(track_id), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Enregistrer l'image avec les boîtes englobantes tracées
            cv2.imwrite(os.path.join(output_path_imgs, f"{i}.jpg"), frame)
            
        
        # Trier les données par ID d'objet (track_id)
        tracks_data.sort(key=lambda x: x[1])

        # Écrire les données triées dans le fichier
        with open(output_file_path, "w") as output_file:
            for data in tracks_data:
                output_file.write(','.join(map(str, data)) + "\n")
        #Ecrire dans TrackEval
        with open(output_file_path_trackEval, "w") as output_file:
            for data in tracks_data:
                output_file.write(','.join(map(str, data)) + "\n") 
         

    

In [13]:
tracking_with_Manu("yolov9c.pt")

---------------------------------
MOT17-02-DPM

0: 384x640 17 persons, 2 benchs, 1 umbrella, 1 handbag, 567.5ms
Speed: 40.1ms preprocess, 567.5ms inference, 26.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 3 benchs, 1 umbrella, 2 handbags, 477.4ms
Speed: 2.1ms preprocess, 477.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 3 benchs, 2 umbrellas, 1 handbag, 466.0ms
Speed: 2.4ms preprocess, 466.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 3 benchs, 1 umbrella, 1 handbag, 462.9ms
Speed: 2.0ms preprocess, 462.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 3 benchs, 1 umbrella, 1 handbag, 467.1ms
Speed: 2.4ms preprocess, 467.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 3 benchs, 1 umbrella, 1 handbag, 462.7ms
Speed: 2.0ms preprocess, 462.7ms inference, 1.0ms postprocess per ima